In [4]:
# importing all needed functions
import os
from astropy.io import fits
import numpy as np
import glob
import io
from contextlib import redirect_stdout
from Tools import tools
from Calculating_det_angles import estimate_source_angles_detectors #importing ma'ams function

# html files to parse
HTML_list = [r"C:\Users\arpan\Downloads\GRB.mhtml",
             r"C:\Users\arpan\Downloads\TGF.mhtml",
             r"C:\Users\arpan\Downloads\SGR.mhtml",
             r"C:\Users\arpan\Downloads\SFLARE.mhtml",
             r"C:\Users\arpan\Downloads\DISTPAR.mhtml",
             r"C:\Users\arpan\Downloads\LOCLPAR.mhtml",
             r"C:\Users\arpan\Downloads\TRANSNT.mhtml",
             r"C:\Users\arpan\Downloads\UNCERT.mhtml"
             ]

event_name = []
event_type = []

for html in HTML_list:
    html_string = tools.extract_strings_html(html)

    # getting only the event names
    for entry in html_string:
        if 'bn' == entry[:2]:
            event_name.append(entry)
            event_type.append(html.split('\\')[-1].split('.')[0])
        
events={"event_name":event_name, "event_type":event_type}

In [6]:
print(len(event_name))

8740


In [17]:
import pandas as pd

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(events)

# Save the DataFrame to a CSV file
df.to_csv('output.csv', index=False)

print("The DataFrame has been saved as 'output.csv'")


The DataFrame has been saved as 'output.csv'


In [11]:
print(20000*7)
print('144445')

140000
144445


In [4]:
import pandas as pd

df = pd.read_csv('all_events.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2932 entries, 0 to 2931
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   event_type   2932 non-null   object 
 1   name         2932 non-null   object 
 2   reliability  2932 non-null   float64
dtypes: float64(1), object(2)
memory usage: 68.8+ KB


In [5]:
import os
dir_path = tools.json_path(r'data_path.json')

tools.create_folder(os.path.join(dir_path, r'100_data_set'))

C:\Users\arpan\OneDrive\Documents\GRB\data\100_data_set
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\100_data_set' and its contents removed successfully.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\100_data_set' created successfully.


In [6]:
df['event_type'].value_counts()

event_type
GRB        1250
SFLARE      796
TGF         668
SGR         114
DISTPAR     104
Name: count, dtype: int64

In [7]:
# Shuffle the DataFrame
df_shuffled = df.sample(frac=1)


In [8]:
event_counter = {'GRB' : 0, 'SFLARE': 0 , 'TGF': 0, 'SGR':0,'DISTPAR':0}
event_limit = 100

event_type, name = [], []
for index, row in df_shuffled.iterrows():
    if event_counter[row['event_type']] < event_limit:
        event_counter[row['event_type']] += 1
        event_type.append(row['event_type'])
        name.append(row['name'])
    

In [9]:
import pandas as pd
data = {'event_type': event_type , 'name': name}
df_100 = pd.DataFrame(data)

df_100.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   event_type  500 non-null    object
 1   name        500 non-null    object
dtypes: object(2)
memory usage: 7.9+ KB


In [ ]:
import os
import shutil

def delete_folders(directory, folder_list):
    for folder in folder_list:
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            try:
                shutil.rmtree(folder_path)
                print(f"Deleted folder: {folder}")
            except Exception as e:
                print(f"Error deleting folder {folder}: {str(e)}")
        else:
            print(f"{folder} is not a folder or does not exist.")

if __name__ == "__main__":
    directory = input("Enter the directory path: ")
    folders = input("Enter the folder names (separated by spaces): ").split()

    delete_folders(directory, folders)

In [4]:
# importing all needed functions
import os
from pathlib import Path
from astropy.io import fits
import numpy as np
import glob
import time
from Calculating_det_angles import estimate_source_angles_detectors  # importing ma'am's function
from Tools import tools
import traceback
from tqdm import tqdm  # for progress bar

import line_profiler
from line_profiler import profile
%load_ext line_profiler

@profile
def process_folder(folder,bin_list):
    try:
        event_type, event = folder.split("_")
        year = '20' + event[2:4] + "/"

        # Use the glob module to search for TTE files in the directory
        target_string = "_tte_"
        file_pattern = str(source_data_set_path / folder / 'current' / f"*{target_string}*")
        NaI_detector = glob.glob(file_pattern)


        # fetching data
        with fits.open(NaI_detector[0], memmap=True) as hdul:
            # energy_channel_data = hdul[1].data
            all_count_data = hdul[2].data
            trigtime = hdul[2].header['TRIGTIME']

        counts = all_count_data['TIME'].astype(float) - trigtime

        # print(type(counts))
        # print(np.array(counts))

        data_array = []

        for i in bin_list:
            # Define the range and number of bins
            range_min = ti[0]
            range_max = ti[-1]
                
            bin_size = i

            # Create bin edges
            bin_edges = np.arange(range_min, range_max, bin_size)

            # Create the histogram using numpy.histogram
            hist, edges = np.histogram(counts, bins=bin_edges)

            hist = hist / bin_size

            data_array = data_array + list(hist)

        data_array = np.array(data_array[:])

        # Save the 2D array to a text file
        data_file_path = data_set_path / f"{event_type}_{event}"
        np.savetxt(data_file_path, data_array, fmt='%d', delimiter='')

    except Exception as e:
        print(f'error {e} in {folder}')
        traceback.print_exc()
        error_folders.append(folder)

# Measure execution time
start_time = time.time()

# name of the data set
source_data_set_path = Path(r"D:\GRB data\500_data_set")


# Get a list of all folders in the specified directory
folders = [str(folder) for folder in os.listdir(source_data_set_path) if os.path.isdir(os.path.join(source_data_set_path, folder))]
# list of bin sizes
bin_list = [0.01, 0.1, 0.5, 1, 5]

# time interval around trigger
ti = [-100,200]
t = ti[1] - ti[0]

# number of datapoints in a light curve
data_no = sum(int(t / (i)) for i in bin_list)
print('number of data points', data_no)

dir_path = Path(tools.json_path(r'data_path.json'))
data_set_name = "dp18-04-2"

# creating the data set folder
data_set_path = dir_path / data_set_name
tools.create_folder(data_set_path)

print('start')
print('total :', len(folders))

# Writing the parameters to a json file
params_dict = {"bin list": bin_list, "time interval": ti, "number of data points": data_no, "data set name": data_set_name, "data set path": str(data_set_path)}
write_json_file = tools.write_json_file(params_dict, data_set_path / 'params.json')

error_folders = []

profiler = line_profiler.LineProfiler()
profiler.add_function(process_folder)
profiler.enable_by_count()

# Process the folders sequentially
for folder in tqdm(folders, desc="Processing folders", unit="folder"):
    process_folder(folder,bin_list)
profiler.print_stats()
print('\n----------------------------------------------------------------------------\n\nevents', folders, ' in folder', data_set_path)
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time:.2f} seconds")

print("errors occured in:",error_folders)

import os
import shutil

def delete_folders(directory, folder_list):
    for folder in folder_list:
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            try:
                shutil.rmtree(folder_path)
                print(f"Deleted folder: {folder}")
            except Exception as e:
                print(f"Error deleting folder {folder}: {str(e)}")
        else:
            print(f"{folder} is not a folder or does not exist.")

folders = error_folders

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
number of data points 33960
C:\Users\arpan\OneDrive\Documents\GRB\data\dp18-04-2
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\dp18-04-2' created successfully.
start
total : 4437
Data has been written to C:\Users\arpan\OneDrive\Documents\GRB\data\dp18-04-2\params.json


Processing folders: 100%|██████████| 4437/4437 [29:28<00:00,  2.51folder/s]  

Timer unit: 1e-07 s

Total time: 1738.23 s
File: C:\Users\arpan\AppData\Local\Temp\ipykernel_4348\2334726758.py
Function: process_folder at line 17

Line #      Hits         Time  Per Hit   % Time  Line Contents
    17                                           @profile
    18                                           def process_folder(folder,bin_list):
    19      4437      48634.0     11.0      0.0      try:
    20      4437     133822.0     30.2      0.0          event_type, event = folder.split("_")
    21      4437     101420.0     22.9      0.0          year = '20' + event[2:4] + "/"
    22                                           
    23                                                   # Use the glob module to search for TTE files in the directory
    24      4437      24857.0      5.6      0.0          target_string = "_tte_"
    25      4437    6218794.0   1401.6      0.0          file_pattern = str(source_data_set_path / folder / 'current' / f"*{target_string}*")
    26    

In [3]:
# importing all needed functions
import os
from pathlib import Path
from astropy.io import fits
import numpy as np
import glob
import time
from Calculating_det_angles import estimate_source_angles_detectors  # importing ma'am's function
from Tools import tools
import traceback
from tqdm import tqdm  # for progress bar
import matplotlib.pyplot as plt

import line_profiler
from line_profiler import profile
%load_ext line_profiler

@profile
def process_folder(folder,bin_list):
    try:
        event_type, event = folder.split("_")
        year = '20' + event[2:4] + "/"

        # Use the glob module to search for TTE files in the directory
        target_string = "_tte_"
        file_pattern = str(source_data_set_path / folder / 'current' / f"*{target_string}*")
        NaI_detector = glob.glob(file_pattern)


        # fetching data
        with fits.open(NaI_detector[0], memmap=True) as hdul:
            # energy_channel_data = hdul[1].data
            all_count_data = hdul[2].data
            trigtime = hdul[2].header['TRIGTIME']

        counts = all_count_data['TIME'].astype(float) - trigtime

        # print(type(counts))
        # print(np.array(counts))

        data_array = []

        for i in bin_list:
            # Define the range and number of bins
            range_min = -data_no * i * r
            range_max =  data_no * i * (1-r)
                
            bin_size = i

            # Create bin edges
            bin_edges = np.arange(range_min, range_max, bin_size)

            # Create the histogram using numpy.histogram
            hist, edges = np.histogram(counts, bins=bin_edges)

            hist = hist / bin_size
            
            # # Plot the histogram
            # plt.figure(figsize=(10,6))
            # plt.plot(edges[1:],hist)
            # # plt.hist(counts, bins=bin_edges,histtype='step', edgecolor='k', alpha=0.7, linewidth=1)
            # plt.xlabel('Time (s)')
            # plt.ylabel('Counts')
            # # bin_size=str(" ;bin size:",float(bin_size))
            # plt.title(f'light curve '+f";bin size = {bin_size}s")

            # # Add a vertical line at x=0
            # plt.axvline(x=0, color='red', linestyle='--',linewidth=0.5)

            # # Show the plot
            # plt.show()

            data_array = data_array + list(hist)

        data_array = np.array(data_array[:])

        # Save the 2D array to a text file
        data_file_path = data_set_path / f"{event_type}_{event}"
        np.savetxt(data_file_path, data_array, fmt='%d', delimiter='')

    except Exception as e:
        print(f'error {e} in {folder}')
        traceback.print_exc()
        error_folders.append(folder)

# Measure execution time
start_time = time.time()

# name of the data set
source_data_set_path = Path(r"D:\GRB data\500_data_set")


# Get a list of all folders in the specified directory
folders = [str(folder) for folder in os.listdir(source_data_set_path) if os.path.isdir(os.path.join(source_data_set_path, folder))]
# list of bin sizes
bin_list = [0.001,0.005,0.01, 0.1, 0.5, 1, 5]

# time interval around trigger
ti = [-100,200]
t = ti[1] - ti[0]

# number of datapoints in a light curve
data_no = 500

# ratio of pre-trigger to post-trigger
r = 0.5

print('number of data points', data_no)

dir_path = Path(tools.json_path(r'data_path.json'))
data_set_name = "dp22-04-1"

# creating the data set folder
data_set_path = dir_path / data_set_name
tools.create_folder(data_set_path)

print('start')
print('total :', len(folders))

# Writing the parameters to a json file
params_dict = {"bin list": bin_list, "time interval": 'n/a', "number of data points": data_no-1, "data set name": data_set_name, "data set path": str(data_set_path)}
write_json_file = tools.write_json_file(params_dict, data_set_path / 'params.json')

error_folders = []

profiler = line_profiler.LineProfiler()
profiler.add_function(process_folder)
profiler.enable_by_count()

c = 0
# Process the folders sequentially
for folder in tqdm(folders, desc="Processing folders", unit="folder"):
    # c += 1
    # if c % 400 == 0:
    #     print(folder)
    process_folder(folder,bin_list)    
    
profiler.print_stats()
print('\n----------------------------------------------------------------------------\n\nevents', folders, ' in folder', data_set_path)
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time:.2f} seconds")

print("errors occured in:",error_folders)

import os
import shutil

def delete_folders(directory, folder_list):
    for folder in folder_list:
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            try:
                shutil.rmtree(folder_path)
                print(f"Deleted folder: {folder}")
            except Exception as e:
                print(f"Error deleting folder {folder}: {str(e)}")
        else:
            print(f"{folder} is not a folder or does not exist.")

folders = error_folders

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
number of data points 500
C:\Users\arpan\OneDrive\Documents\GRB\data\dp22-04-1
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\dp22-04-1' already exists.
delete C:\Users\arpan\OneDrive\Documents\GRB\data\dp22-04-1 y/n?
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\dp22-04-1' and its contents removed successfully.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\dp22-04-1' created successfully.
start
total : 4437
Data has been written to C:\Users\arpan\OneDrive\Documents\GRB\data\dp22-04-1\params.json


Processing folders: 100%|██████████| 4437/4437 [46:37<00:00,  1.59folder/s]  

Timer unit: 1e-07 s

Total time: 2728.1 s
File: C:\Users\arpan\AppData\Local\Temp\ipykernel_13152\4227725635.py
Function: process_folder at line 18

Line #      Hits         Time  Per Hit   % Time  Line Contents
    18                                           @profile
    19                                           def process_folder(folder,bin_list):
    20      4437     101429.0     22.9      0.0      try:
    21      4437     280111.0     63.1      0.0          event_type, event = folder.split("_")
    22      4437     232146.0     52.3      0.0          year = '20' + event[2:4] + "/"
    23                                           
    24                                                   # Use the glob module to search for TTE files in the directory
    25      4437      60180.0     13.6      0.0          target_string = "_tte_"
    26      4437   13550238.0   3053.9      0.0          file_pattern = str(source_data_set_path / folder / 'current' / f"*{target_string}*")
    27    

In [ ]:
plt.show()

In [8]:
import os
import json

def find_json_file(directory):
    """
    Find a JSON file in a directory (and its subdirectories) and load it into a dictionary.
    
    Parameters:
        directory (str): Path to the directory to search.
    
    Returns:
        dict or None: Dictionary containing the JSON data if a file is found, 
                      None otherwise.
    """
    for root, dirs, files in os.walk(directory):
        for file_name in files:
            if file_name.endswith('.json'):
                file_path = os.path.join(root, file_name)
                with open(file_path, 'r') as file:
                    try:
                        json_data = json.load(file)
                        return json_data
                    except json.JSONDecodeError:
                        print(f"Error: JSON decode failed for file '{file_path}'")
    print("No JSON file found in the directory.")
    return None

# # Example usage:
# directory_path = '/path/to/directory'  # Replace with the path to your directory
# json_data = find_json_file(directory_path)
# if json_data:
#     print(json_data)

import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import json
# importing data
# Read the 2D array back from the text file
dir_path = r"C:\Users\arpan\OneDrive\Documents\GRB\data\dp19-04-1"         #dp1-1set/dp1-1
event_types = {1:'GRB',2:'TGF',3:'SGR',4:'SFLARE'}
event_counter = {'GRB':0,'TGF':0,'SGR':0,'SFLARE':0}
# Replace 'your_search_string' with the string you are looking for in file names
search_string = 'bn'
search_pattern = os.path.join(dir_path, f'*{search_string}*')
matching_files = glob.glob(search_pattern)
json_data = find_json_file(dir_path)
print(json_data)

X = []
data_no = json_data['number of data points']
print(data_no)

Y = []

for file in matching_files:
    event_type , event_name = file.split('\\')[-1].split('_')
    # print(event_type,event_name)
    loaded_data = np.loadtxt(file, delimiter='\t').astype(np.int32)

    if event_type not in event_counter.keys():
        continue
    
    if event_counter[event_type] > 1200:
        continue
    
    y = [0,0,0,0]
    
    for key,value in event_types.items():
        if value in file.split('\\')[-1]:
            y[key-1] = 1
            Y.append(y)
            event_counter[value] += 1
            sgrf = 1 if key == 3 else 0
            if sgrf == 1:
                Y.append(y)
                event_counter[value] += 1
            break
    
    f = 0
    x = []
    for k in json_data['bin list']:
        x.append(loaded_data[f:f+data_no])
        f = f + data_no
    X.append(np.array(x))
    if sgrf == 1:
        f = 0
        x = []
        for k in json_data['bin list']:
            x.append(loaded_data[f:f+data_no])
            f = f + data_no
        X.append(np.array(x))

X = np.array(X)
Y = np.array(Y)

X_shape = X.shape + tuple(range(1,2))
Y_shape = Y.shape + tuple(range(1,2))

X = X.reshape(X_shape)
Y = Y.reshape(Y_shape)

# Split the train and the validation set for the fitting
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.3)#, random_state=2)
print("X_train shape",X_train.shape)
print("X_test shape",X_val.shape)
print("Y_train shape",Y_train.shape)
print("Y_test shape",Y_val.shape)
print(event_counter)

{'bin list': [0.001, 0.005, 0.01, 0.1, 0.5, 1, 5], 'time interval': 'n/a', 'number of data points': 999, 'data set name': 'dp19-04-1', 'data set path': 'C:\\Users\\arpan\\OneDrive\\Documents\\GRB\\data\\dp19-04-1'}
999
(4377, 7, 999)
(4377, 4)


TypeError: 'tuple' object cannot be interpreted as an integer

In [25]:
print(type(X.shape))
print(type(Y.shape))

X_shape = X.shape + tuple(range(1,2))
Y_shape = Y.shape + tuple(range(1,2))

X = X.reshape(X_shape)
Y = Y.reshape(Y_shape)

# Split the train and the validation set for the fitting
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.3)#, random_state=2)
print("X_train shape",X_train.shape)
print("X_test shape",X_val.shape)
print("Y_train shape",Y_train.shape)
print("Y_test shape",Y_val.shape)
print(event_counter)

<class 'tuple'>
<class 'tuple'>
(4377, 7, 999, 1)
(4377, 4, 1)
X_train shape (3063, 7, 999, 1)
X_test shape (1314, 7, 999, 1)
Y_train shape (3063, 4, 1)
Y_test shape (1314, 4, 1)
{'GRB': 1201, 'TGF': 1139, 'SGR': 978, 'SFLARE': 1059}


In [12]:
print(X[1])

[[3000    0 2000 ... 2000 2000 1000]
 [1000 1200  400 ... 1400 2200 1400]
 [1500 1900 1500 ...  700 1500 1000]
 ...
 [   0    0    0 ... 1222 1248 1314]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]


In [2]:
print(len(error_folders))

0


In [4]:
delete_folders(source_data_set_path, error_folders)

Deleted folder: DISTPAR_bn161026708
Deleted folder: DISTPAR_bn170908610
Deleted folder: GRB_bn081122520
Deleted folder: GRB_bn081122614
Deleted folder: GRB_bn081130629
Deleted folder: GRB_bn090225009
Deleted folder: GRB_bn090301315
Deleted folder: GRB_bn090425377
Deleted folder: GRB_bn090522344
Deleted folder: GRB_bn090529310
Deleted folder: GRB_bn090621185
Deleted folder: GRB_bn090621922
Deleted folder: GRB_bn090717111
Deleted folder: GRB_bn090730608
Deleted folder: GRB_bn090815438
Deleted folder: GRB_bn090820027
Deleted folder: GRB_bn090824918
Deleted folder: GRB_bn090904581
Deleted folder: GRB_bn090922539
Deleted folder: GRB_bn091023021
Deleted folder: GRB_bn091024372
Deleted folder: GRB_bn091030613
Deleted folder: GRB_bn091106762
Deleted folder: GRB_bn091123081
Deleted folder: GRB_bn091223191
Deleted folder: GRB_bn091227294
Deleted folder: GRB_bn100225249
Deleted folder: GRB_bn100304534
Deleted folder: GRB_bn100318611
Deleted folder: GRB_bn100326402
Deleted folder: GRB_bn100411516
